In [6]:
import pandas as pd
import spacy
import en_core_web_sm
from collections import Counter

In [7]:
df = pd.read_csv("./downloads/reviews100.csv")
df

,stars,comment
0,4.0 out of 5 stars,I purchased this bad boy for my wife’s home of...
1,4.0 out of 5 stars,This is a quite nice machine. Very basic yet b...
2,4.0 out of 5 stars,PROS: I want to cover first the high-quality a...
3,4.0 out of 5 stars,Last it lasted 6 years! And this one cost $4 l...
4,5.0 out of 5 stars,"Update: After over a year, the shredder is sti..."
...,...,...
95,4.0 out of 5 stars,Very good value for this Amazon Basic cross cu...
96,5.0 out of 5 stars,This is a great machine at an even better pric...
97,1.0 out of 5 stars,"So I decided I wanted to get a paper shredder,..."
98,1.0 out of 5 stars,I have now been through 3 of these shredders. ...


In [8]:
df['comment'] = df['comment'].apply(lambda x: x.strip())
#df = df.sort_values(['stars'], ascending=False).iloc[0:30] # get 30 top reviews 
df = df.sample(frac=1).iloc[0:30] # get 30 top reviews 
df.to_csv("30_reviews.csv", index=False) # save to csv for manual review
df

,stars,comment
97,1.0 out of 5 stars,"So I decided I wanted to get a paper shredder,..."
11,1.0 out of 5 stars,I order a lot from Amazon for convenience but ...
64,5.0 out of 5 stars,Pretty thrilled with this shredder right now !...
98,1.0 out of 5 stars,I have now been through 3 of these shredders. ...
4,5.0 out of 5 stars,"Update: After over a year, the shredder is sti..."
10,1.0 out of 5 stars,"SAYS it's made for home office, I guess becau..."
40,5.0 out of 5 stars,Quiet machine with ample capacity both in numb...
41,1.0 out of 5 stars,Very disappointed. Received shredder and plugg...
88,5.0 out of 5 stars,The Shredder is listed as a light/medium duty ...
76,3.0 out of 5 stars,"Will only give it 3 stars, since it arrived br..."


In [19]:
# nlp = spacy.load("en_core_web_sm") # buggy on ipynb
nlp = en_core_web_sm.load() # for ipynb
noun_adj_pairs = {}
# print(df['comment'])
for comment in enumerate(df['comment']):
    # print(comment[1])
    doc = nlp(str(comment[1])) # change tuple to str for nlp
    doc_len = len(doc)
    for i, token in enumerate(doc): #token has TEXT, LEMMA, POS, TAG, DEP, SHAPE, ALPHA, STOP
        if token.pos_ not in ('NOUN', 'PROPN'):
            continue
        for j in range(i+1, doc_len): # check for pairs after identified noun/pnoun
            if doc[j].pos_ == 'ADJ':
                pair = (str(token), str(doc[j])) # change to str. ele in dict keys has to be hashable even inside tuples
                # check if already exist in dictionary
                if pair in noun_adj_pairs:
                    noun_adj_pairs[pair] += 1
                else:
                    noun_adj_pairs[pair] = 1
top_10_noun_adj_pairs = dict(Counter(noun_adj_pairs).most_common(5))
top_10_noun_adj_pairs
# top_10_noun_adj_pairs = {key: val for key, val in sorted(noun_adj_pairs.items(), key=lambda item: item[1])}
# print(noun_adj_pairs.keys())
# noun_adj_pairs = dict(sorted(noun_adj_pairs.items()))
# print(noun_adj_pairs)

{('shredder', 'small'): 36,
 ('shredder', 'little'): 34,
 ('shredder', 'few'): 32,
 ('shredder', 'full'): 21,
 ('shredder', 'sure'): 21}